# Introduction

Python의 Model I/O 에 대해 학습함

FewShotPromptTemplate

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

chat = ChatOpenAI(
    model = "gpt-4.1-nano",
    temperature = 0.1,
    streaming = True,
    callbacks = [
        StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


/var/folders/h7/4cv0q0kj0bq39hdg5q154dbw0000gn/T/ipykernel_18379/1151379228.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(


In [3]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

In [4]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

In [5]:
chain = prompt | chat

chain.invoke({"country" : "Spain"})

I know this:
- Capital: Madrid
- Language: Spanish
- Food: Tapas and Paella
- Currency: Euro

AIMessage(content='I know this:\n- Capital: Madrid\n- Language: Spanish\n- Food: Tapas and Paella\n- Currency: Euro', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--7fcb8751-60ac-4dab-9f56-c56496641149-0')

# FewShotChatMessagePromptTemplate

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    model="gpt-4o-mini",         # Specifies the OpenAI model
    temperature=0.1,             # Low randomness; more deterministic responses
    streaming=True,              # Enables streaming responses token-by-token
    callbacks=[
        StreamingStdOutCallbackHandler()  # Streams output to standard output
    ],
)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [7]:
example_prompt: ChatPromptTemplate = ChatPromptTemplate.from_messages(
    [
        ("human", "what do you know about {country}?"),
        ("ai", "{answer}")
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt: ChatPromptTemplate = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a geography expert, you give short answers."),
        example_prompt,
        ("human", "what do you know abiut {country}")
    ]
)

In [8]:
chain: any= final_prompt | chat
chain.invoke(
    {
        "country", "korea"
    }
)

Korea refers to two countries: 

1. **South Korea** (Republic of Korea)
   - Capital: Seoul
   - Language: Korean
   - Currency: South Korean Won (KRW)

2. **North Korea** (Democratic People's Republic of Korea)
   - Capital: Pyongyang
   - Language: Korean
   - Currency: North Korean Won (KPW)

Both countries share a common cultural and historical background but have distinct political systems.

AIMessage(content="Korea refers to two countries: \n\n1. **South Korea** (Republic of Korea)\n   - Capital: Seoul\n   - Language: Korean\n   - Currency: South Korean Won (KRW)\n\n2. **North Korea** (Democratic People's Republic of Korea)\n   - Capital: Pyongyang\n   - Language: Korean\n   - Currency: North Korean Won (KPW)\n\nBoth countries share a common cultural and historical background but have distinct political systems.", additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--9c9edb66-77db-4be1-85fe-95c42516f5c1-0')

# LengthBasedExampleSelector

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    model = "gpt-4.1-nano",
    temperature = 0.1,
    streaming = True,
    callbacks = [
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [10]:
class RandomExampleSelecter(BaseExampleSelector):
    def __init__(self, examples):
        self.examples: Any = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]

In [11]:
example_prompt = PromptTemplate = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector: RandomExampleSelecter = RandomExampleSelecter(
    examples=examples
)

In [12]:
prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector = example_selector,
    suffix="Human: What do you know about {counrty}?",
    input_variables=["country"],
)
prompt.format(counrty="Brazil")

'Human: What do you know about Italy?\nAI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

# Serialization and Composition

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompt import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat: Any = ChatOpenAI(
    model="gpt-4o-mini",         # Specifies the OpenAI model
    temperature=0.1,             # Low randomness; more deterministic responses
    streaming=True,              # Enables streaming responses token-by-token
    callbacks=[
        StreamingStdOutCallbackHandler()  # Streams output to standard output
    ],
)

ModuleNotFoundError: No module named 'langchain.prompt'

In [ ]:
intro: promptTemplate = PromptTemplate.from_template(
    """
    You are a role playing assistant
    And you are impersomating a {character}
    """
)

example: promptTemplate = PromptTemplate.from_template(
    """
    this is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
    """
)

start: promptTemplate = PromptTemplate.from_template(
    """
    start now:
    
    Human: {question}
    you:
    """
)

final: Prompttemplate = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
    """
)

prompt: list[tuple[str, PromptTempl... ]] = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]

full_prompt = pipelinePromptTemplate(
    final_prompt = final,
    pipeline_primpts = prompts
)

chain: Any = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)